In [1]:
from time import time
import pandas as pd
import numpy as np
from PIL import Image
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D,concatenate
from keras.layers import UpSampling1D, Reshape, Lambda, Flatten, Activation,Concatenate
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

In [2]:
train_data = np.load('train_data.npy',)
train_label = np.load('train_label.npy',)
test_data = np.load('test_data.npy',)
test_label = np.load('test_label.npy',)

In [3]:
noramlized_train_data = (train_data - train_data.mean())/train_data.std()
normalized_test_data = (test_data - test_data.mean())/test_data.std()

In [4]:
train_data_real = train_data.real
train_data_imag = train_data.imag
train_data_ext = np.hstack((train_data_real,train_data_imag))
print(train_data_ext.shape)

test_data_real = test_data.real
test_data_imag = test_data.imag
test_data_ext = np.hstack((test_data_real,test_data_imag))
print(test_data_ext.shape)



(18000, 16376)
(18000, 16376)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 64
# trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# train_set = datasets.MNIST('../data', train=True, transform=trans, download=True)
# test_set = datasets.MNIST('../data', train=False, transform=trans, download=True)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle=True)

class AENet(nn.Module):
    def __init__(self):
        super(AENet, self).__init__()
        # encoder:
        self.encoder_fc1 = nn.Linear(8188*2, 500)
        self.encoder_fc2 = nn.Linear(500, 64)
        self.encoder_fc3 = nn.Linear(64,30)
        
        self.decoder_fc1 = nn.Linear(30, 64)
        self.decoder_fc2 = nn.Linear(64, 500)
        self.decoder_fc3 = nn.Linear(500, 8188*2)
        
        self.encoder = nn.Sequential(
            nn.Linear(8188*2, 1000),
            nn.ReLU(),
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 30),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(30, 256),
            nn.ReLU(),
            nn.Linear(256, 1000),
            nn.ReLU(),
            nn.Linear(1000, 8188*2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [6]:
train_data_ext.shape

(18000, 16376)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
for HIDDEN_NUM in [64]:
    for TRAINING_EPOCH in [50]:
        print('training epoch',TRAINING_EPOCH)
        batch_size = 64
        class AENet(nn.Module):
            def __init__(self):
                super(AENet, self).__init__()
                self.encoder = nn.Sequential(
                    nn.Linear(8188*2, 1000),
                    nn.ReLU(),
                    nn.Linear(1000, 256),
                    nn.ReLU(),
                    nn.Linear(256, HIDDEN_NUM),
                    nn.ReLU()
                )
                self.decoder = nn.Sequential(
                    nn.Linear(HIDDEN_NUM, 256),
                    nn.ReLU(),
                    nn.Linear(256, 1000),
                    nn.ReLU(),
                    nn.Linear(1000, 8188*2),
                    nn.Sigmoid()
                )

            def forward(self, x):
                x = self.encoder(x)
                x = self.decoder(x)
                return x

        
        rolling_results_logger = []
        
        for rolling_index in range(36):
            rolling_training_idx_start = rolling_index * 10
            rolling_training_idx_end = rolling_index * 10 + 40
            rolling_testing_idx_start = rolling_index * 10 + 40
            rolling_testing_idx_end = rolling_index * 10 + 50

            # generating training dataset and the testing dataset
            # training set shape of 60, testing set of shape 10.
            train_set_this_time = []
            test_set_this_time = []
            for gen_i in range(45):
                train_set_this_time.extend(train_data_ext[gen_i * 400: (gen_i + 1) * 400][rolling_training_idx_start: rolling_training_idx_end]) 
                test_set_this_time.extend(train_data_ext[gen_i * 400: (gen_i + 1) * 400][rolling_testing_idx_start:rolling_testing_idx_end])
            train_set_this_time = np.asarray(train_set_this_time)
            test_set_this_time = np.asarray(test_set_this_time)
            
            # generate a new model
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = AENet().to(device)
            optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

            def train(model, device, optimizer, epoch , prt):
                model.train()
                for i in range(int(45*40/batch_size)):
                    sample_idx = np.random.randint(0,45*40,batch_size)
                    data, target = torch.as_tensor(train_set_this_time[sample_idx]).float(), torch.as_tensor(train_set_this_time[sample_idx]).float()
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()
                    output = model(data)
                    loss = F.mse_loss(output, target)
                    loss.backward()
                    optimizer.step()
                    if prt and (i) % 1000 == 0:
                        print('Train Epoch: {:3} \tLoss: {:.6f}'.format(
                            epoch,
                            loss.item())
                        )

            for epoch in range(TRAINING_EPOCH):
                if (epoch) %10 ==0:
                    train(model, device, optimizer, epoch, prt = True)
                else:
                    train(model, device, optimizer, epoch, prt = False)


            # start evaluating the model.
            model.eval()
            hidden_train = model.encoder(torch.as_tensor(train_set_this_time).float().to(device))
            hidden_test = model.encoder(torch.as_tensor(test_set_this_time).float().to(device))
            
            # plot the clustering results (using curves.)
            dist_tot_list = []
            for j in range(10):
                dist_list = []

                for i in range(45):
                    dist_list.append(torch.norm(hidden_test[i*10: (i+1)*10][j] - hidden_train[22*40: 23*40][0]).cpu().detach())
                dist_tot_list.append(dist_list)


            dist_tot_list = np.asarray(dist_tot_list)
            threshold_result_list = []
            for threshold in [np.mean(dist_tot_list,0)[22] + j*np.std(dist_tot_list,0)[22] for j in range(20)]:
                f1_list = []
#                 for i in range(45):
#                     temp_tp = np.sum(dist_tot_list[:,22]<threshold)/10
#                     temp_fn = np.sum(dist_tot_list[:,22]>threshold)/10
#                     if i == 22:

#                         temp_fp = (np.sum(dist_tot_list[:,:22]<threshold) + np.sum(dist_tot_list[:,23:]<threshold) )/440
#                         #print('22',temp_fp)
#                     else:
#                         temp_fp = np.sum(dist_tot_list[:,i]<threshold)/10

#                     f1_score_tmp = temp_tp/(temp_tp+0.5*(temp_fp+temp_fn))
#                     f1_list.append(f1_score_tmp)
                temp_tp = np.sum(dist_tot_list[:,22]<threshold)/10
                temp_fn = np.sum(dist_tot_list[:,22]>threshold)/10
                temp_fp = (np.sum(dist_tot_list[:,:22]<threshold) + np.sum(dist_tot_list[:,23:]<threshold) )/440
                
                f1_score_tmp = temp_tp/(temp_tp+0.5*(temp_fp+temp_fn))
                f1_list.append(f1_score_tmp)
                
                
                print('threshold now',threshold, np.mean(f1_list))
                threshold_result_list.append(np.mean(f1_list))
            
            
            tf.config.experimental_run_functions_eagerly(True)
            """VAE"""
            net_3_input= Input(shape=(HIDDEN_NUM,))
            h_1 = Dense(HIDDEN_NUM, activation='tanh')(net_3_input)
            HDNUM = 30
            z_mean = Dense(HDNUM)(h_1)
            z_log_var = Dense(HDNUM)(h_1)
            z_mean_2 = Dense(HDNUM)(h_1)
            z_log_var_2 = Dense(HDNUM)(h_1)
            z_mean_con = Concatenate()([z_mean,z_mean_2])
            def sampling1d(args):   
                z_mean, z_log_var = args  
                epsilon = K.random_normal(shape=(40, HDNUM), mean=0.,stddev=0.1)
                return z_mean + K.exp(z_log_var / 2)* epsilon
            def sampling5d(args):   
                z_mean, z_log_var = args  
                epsilon = K.random_normal(shape=(40, HDNUM), mean=0.,stddev=0.1)
                return z_mean + K.exp(z_log_var / 2)* epsilon

            z = Lambda(sampling1d, output_shape=(HDNUM,))([z_mean, z_log_var])
            z_2 = Lambda(sampling5d, output_shape=(HDNUM,))([z_mean_2, z_log_var_2])
            z_con = Concatenate()([z,z_2])
            decoded = Dense(HIDDEN_NUM, activation='tanh')(z_con)

            def vae_loss(x, decoded):
                xent_loss = K.sum((objectives.mse(x ,decoded)),axis=-1)
                #kl_loss_d1 = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) 
                m = K.constant(0.4)
                s = K.constant(1)
                #kl_loss_d1 = K.sum(K.log(2/K.exp(z_log_var/2))+(K.square(z_mean)+(K.exp(z_log_var/2)-K.constant(1))*(K.exp(z_log_var/2)+K.constant(1)))/(K.constant(2)),axis = -1)
                kl_loss_d1 = K.sum(K.log(2*s/K.exp(z_log_var/2))+(K.constant(2)*m*(-K.exp(-(K.square(z_mean))/((K.constant(2))*K.exp(z_log_var)))*K.exp(z_log_var/2) + K.sqrt(K.constant(np.pi/2))*z_mean*(K.constant(1)-K.tanh(K.constant(1.19)*z_mean/K.constant(np.sqrt(2))/K.exp(z_log_var/2)))) )/(K.square(s))+(K.square(m-z_mean)+(K.exp(z_log_var/2)-s)*(K.exp(z_log_var/2)+s))/(K.constant(2)*K.square(s)),axis = -1)
                return 1*xent_loss + 0.2*kl_loss_d1 
            EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
            vae = Model(net_3_input, decoded)
            vae.compile(optimizer='rmsprop', loss=vae_loss,experimental_run_tf_function=False)

            vae.fit(hidden_train.cpu().detach().numpy().astype(float), hidden_train.cpu().detach().numpy().astype(float),
                shuffle=True,
                epochs=1,
                verbose=1,
                batch_size=40,
                validation_split=0.2,callbacks=[EarlyStopping])
            vis = Model(net_3_input, z_mean_con)


            hidden_train_cas = vis.predict(hidden_train.cpu().detach().numpy().astype(float),batch_size=100)
            hidden_test_cas = vis.predict(hidden_test.cpu().detach().numpy().astype(float),batch_size=100)

            # plot the clustering results (using curves.)
            dist_tot_list = []
            for j in range(10):
                dist_list = []

                for i in range(45):
                    dist_list.append(np.linalg.norm(hidden_test_cas[i*10: (i+1)*10,:HDNUM][j] - hidden_train_cas[22*40: 23*40,:HDNUM][0]))
                dist_tot_list.append(dist_list)


            dist_tot_list = np.asarray(dist_tot_list)
            threshold_result_list_2 = []
            for threshold in [np.mean(dist_tot_list,0)[22] + j*np.std(dist_tot_list,0)[22] for j in range(20)]:
                f1_list_2 = []

                temp_tp = np.sum(dist_tot_list[:,22]<threshold)/10
                temp_fn = np.sum(dist_tot_list[:,22]>threshold)/10
                temp_fp = (np.sum(dist_tot_list[:,:22]<threshold) + np.sum(dist_tot_list[:,23:]<threshold) )/440

                f1_score_tmp = temp_tp/(temp_tp+0.5*(temp_fp+temp_fn))
                f1_list_2.append(f1_score_tmp)


                print('threshold now (cas)',threshold, np.mean(f1_list_2))

                threshold_result_list_2.append(np.mean(f1_list_2))

            rolling_results_logger.append((threshold_result_list, threshold_result_list_2))
        np.save(f'results/rolling_results_logger_{HIDDEN_NUM}_{TRAINING_EPOCH}.npy',rolling_results_logger)


training epoch 50
Train Epoch:   0 	Loss: 0.250048
Train Epoch:  10 	Loss: 0.247121
Train Epoch:  20 	Loss: 0.233220
Train Epoch:  30 	Loss: 0.000399
Train Epoch:  40 	Loss: 0.000089
threshold now 5.252771188679617e-06 0.5827814569536424
threshold now 5.729660472297837e-06 0.7874720357941836
threshold now 6.206549755916058e-06 0.8897876643073812
threshold now 6.683439039534278e-06 0.8800000000000001
threshold now 7.160328323152498e-06 0.8535402521823472
threshold now 7.637217606770719e-06 0.8388941849380361
threshold now 8.114106890388939e-06 0.8349146110056925
threshold now 8.59099617400716e-06 0.8309726156751652
threshold now 9.06788545762538e-06 0.8262910798122066
threshold now 9.5447747412436e-06 0.819366852886406
threshold now 1.002166402486182e-05 0.8110599078341014
threshold now 1.0498553308480041e-05 0.7971014492753624
threshold now 1.0975442592098261e-05 0.7864164432529044
threshold now 1.1452331875716482e-05 0.7822222222222223
threshold now 1.1929221159334702e-05 0.7739665787

In [63]:
f1_list

[0.859375]

In [64]:
np.asarray(rolling_results_logger).shape

(36, 20)

In [66]:
np.mean(rolling_results_logger,0)

array([0.60680818, 0.84045906, 0.90587455, 0.91117211, 0.90557849,
       0.89963353, 0.89234407, 0.88600267, 0.88108383, 0.87594259,
       0.8701446 , 0.86454549, 0.85984115, 0.85481892, 0.84969614,
       0.84616268, 0.84205252, 0.8373118 , 0.83247875, 0.82807637])

In [68]:
from time import time
import pandas as pd
import numpy as np
from PIL import Image
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D,concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

In [69]:
hidden_test.shape

torch.Size([450, 32])

In [88]:
hidden_train.shape

torch.Size([1800, 32])

In [89]:
hidden_train.cpu().detach().numpy().shape

(1800, 32)

In [136]:
tf.config.experimental_run_functions_eagerly(True)
"""VAE"""
net_3_input= Input(shape=(32,))
h_1 = Dense(32, activation='tanh')(net_3_input)
HDNUM = 5
z_mean = Dense(HDNUM)(h_1)
z_log_var = Dense(HDNUM)(h_1)
z_mean_2 = Dense(HDNUM)(h_1)
z_log_var_2 = Dense(HDNUM)(h_1)
z_mean_con = Concatenate()([z_mean,z_mean_2])
def sampling1d(args):   
    z_mean, z_log_var = args  
    epsilon = K.random_normal(shape=(40, HDNUM), mean=0.,stddev=0.1)
    return z_mean + K.exp(z_log_var / 2)* epsilon
def sampling5d(args):   
    z_mean, z_log_var = args  
    epsilon = K.random_normal(shape=(40, HDNUM), mean=0.,stddev=0.1)
    return z_mean + K.exp(z_log_var / 2)* epsilon

z = Lambda(sampling1d, output_shape=(HDNUM,))([z_mean, z_log_var])
z_2 = Lambda(sampling5d, output_shape=(HDNUM,))([z_mean_2, z_log_var_2])
z_con = Concatenate()([z,z_2])
decoded = Dense(32, activation='tanh')(z_con)

def vae_loss(x, decoded):
    xent_loss = K.sum((objectives.mse(x ,decoded)),axis=-1)
    #kl_loss_d1 = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) 
    m = K.constant(0.4)
    s = K.constant(1)
    #kl_loss_d1 = K.sum(K.log(2/K.exp(z_log_var/2))+(K.square(z_mean)+(K.exp(z_log_var/2)-K.constant(1))*(K.exp(z_log_var/2)+K.constant(1)))/(K.constant(2)),axis = -1)
    kl_loss_d1 = K.sum(K.log(2*s/K.exp(z_log_var/2))+(K.constant(2)*m*(-K.exp(-(K.square(z_mean))/((K.constant(2))*K.exp(z_log_var)))*K.exp(z_log_var/2) + K.sqrt(K.constant(np.pi/2))*z_mean*(K.constant(1)-K.tanh(K.constant(1.19)*z_mean/K.constant(np.sqrt(2))/K.exp(z_log_var/2)))) )/(K.square(s))+(K.square(m-z_mean)+(K.exp(z_log_var/2)-s)*(K.exp(z_log_var/2)+s))/(K.constant(2)*K.square(s)),axis = -1)
    return 1*xent_loss + 0.2*kl_loss_d1 
EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
vae = Model(net_3_input, decoded)
vae.compile(optimizer='rmsprop', loss=vae_loss,experimental_run_tf_function=False)

vae.fit(hidden_train.cpu().detach().numpy().astype(float), hidden_train.cpu().detach().numpy().astype(float),
    shuffle=True,
    epochs=1,
    verbose=1,
    batch_size=40,
    validation_split=0.2,callbacks=[EarlyStopping])
vis = Model(net_3_input, z_mean_con)


hidden_train_cas = vis.predict(hidden_train.cpu().detach().numpy().astype(float),batch_size=100)
hidden_test_cas = vis.predict(hidden_test.cpu().detach().numpy().astype(float),batch_size=100)

# plot the clustering results (using curves.)
dist_tot_list = []
for j in range(10):
    dist_list = []

    for i in range(45):
        dist_list.append(np.linalg.norm(hidden_test_cas[i*10: (i+1)*10,:HDNUM][j] - hidden_train_cas[22*40: 23*40,:HDNUM][0]))
    dist_tot_list.append(dist_list)


dist_tot_list = np.asarray(dist_tot_list)
threshold_result_list = []
for threshold in [np.mean(dist_tot_list,0)[22] + j*np.std(dist_tot_list,0)[22] for j in range(20)]:
    f1_list = []

    temp_tp = np.sum(dist_tot_list[:,22]<threshold)/10
    temp_fn = np.sum(dist_tot_list[:,22]>threshold)/10
    temp_fp = (np.sum(dist_tot_list[:,:22]<threshold) + np.sum(dist_tot_list[:,23:]<threshold) )/440

    f1_score_tmp = temp_tp/(temp_tp+0.5*(temp_fp+temp_fn))
    f1_list.append(f1_score_tmp)


    print('threshold now',threshold, np.mean(f1_list))

36/36 [==============================] - 21s 584ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00


threshold now 2.9573590154541307e-07 0.6646525679758308
threshold now 3.8174605521135163e-07 0.9306698002350177
threshold now 4.677562088772902e-07 0.9492988133764833
threshold now 5.537663625432288e-07 0.9044193216855088
threshold now 6.397765162091673e-07 0.854368932038835
threshold now 7.257866698751059e-07 0.8201304753028892
threshold now 8.117968235410444e-07 0.7949412827461608
threshold now 8.97806977206983e-07 0.7726075504828797
threshold now 9.838171308729216e-07 0.7665505226480835
threshold now 1.0698272845388601e-06 0.7612456747404844
threshold now 1.1558374382047987e-06 0.7534246575342466
threshold now 1.2418475918707372e-06 0.7470288624787776
threshold now 1.3278577455366758e-06 0.738255033557047
threshold now 1.4138678992026144e-06 0.7308970099667774
threshold now 1.499878052868553e-06 0.7284768211920529
threshold now 1.5858882065344915e-06 0.723089564502876
threshold now 1.67189836020043e-06 0.7183673469387755
threshold now 1.7579085138663686e-06 0.7108239095315024
thresh

In [ ]:
# display a 2D plot of the digit classes in the latent space  
# x_vis = vis.predict(Zero_one_normalized_encoded_variables[lower_lim:upper_lim], batch_size=batch_size)

test_encoded_variables = encoder.predict(test_Image_RGB[:10000],batch_size=100)
vis = Model(input=net_3_input, output=z_mean_con)
x_vis = vis.predict(test_encoded_variables[:10000], batch_size=batch_size)

x_vis_1 = vis.predict(test_encoded_variables[:10000], batch_size=batch_size)  
x_vis_1[Class[:10000]<1]=0

x_vis_2 = vis.predict(test_encoded_variables[:10000], batch_size=batch_size)  
x_vis_2[Class[:10000]==1]=0


In [90]:
epsilon

NameError: name 'epsilon' is not defined